## Question 1 (Code  2, 3)

Student 宿永烨 18342107

### Code 2

In [ ]:
def findClosestCentroids(X, centroids):
# 该方法为每个数据点计算其距离最近的质心
# 返回值为每个数据点距离最近的质心idx
# idx是一个m * 1的向量，其中m为数据集中的数据数目

	# 簇的数目K
	K = centroids.shape[0]

	# 返回值
	idx = np.zeros((X.shape[0], 1));

#====================== YOUR CODE HERE ======================
# 
#  计算每个训练数据距离最近的质心，并且将其位置存在idx中
#  返回值idx中，idx(i)应等于数据i最近的质心，因此idx(i)的值应在1到K之间
#  Answer:
    for i in range(X.shape[0]):
        min_distance = 9.9E+10
        for j in range(K):
            if min_distance == min(min_distance,np.linalg.norm(X[i]-centroids[j])):
                idx[i] = j
    return idx
#==========================================================



### Code 3

In [ ]:
def computeCentroids(X, idx, K):
# 通过计算均值返回新的质心位置
# 输入值为数据集X（每行是一个数据点），质心1到k的位置idx，质心数量K
# 返回值为centroids，其每一行为一个质心的位置（该簇的所有点位置的均值）

	# 数据集行列数
	m, n = X.shape

	# 返回值
	centroids = np.zeros((K, n));

	#====================YOUR CODE HERE ======================
	# 遍历每个簇并计算其位置均值
	# 返回值中centroids[i]应为i簇的所有数据点位置均值
	# Answer
    centroids_match_count = np.zeros((K,1)) #this is a 1 dimensional vector that counts the matching points' number. 
    # calculating the sum of each centroids' matching points' coordinate.
    for i in range(m):
        centroids[idx[i]] += X[i]
        centroids_match_count[idx[i]] += 1;
    for i in range(K):
        centroids[i] /= centroids_match_count[i];
    return centroids
	#=======================================================


## Question 2(Code 4)

In [ ]:

from numpy import *

# 训练数据
def loadDataSet():
    # 共六条训练文本句子
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    # 分类标签
    classVec = [0,1,0,1,0,1]    #1为侮辱性句子，0为非侮辱性句子
    return postingList,classVec

# 创建一个不重复的词汇集合
def createVocabList(dataSet):
    vocabSet = set([])  #创建空集
    for document in dataSet:
        vocabSet = vocabSet | set(document) #两集合的并集
    return list(vocabSet)

# 记录一行句子中每个词汇集合中的词汇出现与否，返回一维01数组，其长度等于词汇集合长度
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
    return returnVec

# 朴素贝叶斯分类器训练方法
# 输入为训练文本数据trainMatrix和训练文本数据分类trainCategory
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
# =========YOUR CODE HERE==========
#
#	在此计算得出概率数组p0Vect和p1Vect
    p0Vect = [0]*numWords
    p1Vect = [0]*numWords
    #Here starts getting p0Vect
    for i in range(numWords):
        total_num_of_word = 0  # total number count of a specific word in trainMatrix
        non_abuse_case_num = 0 # total number count of a specific word in trainMatrix in non-abuse conditions
        for j in range(numTrainDocs):
            total_num_of_word += trainMatrix[j][i]
            if trainCategory[j]==0:
                non_abuse_case_num += trainMatrix[j][i]
        p0Vect[i] = (1-pAbusive)*numTrainDocs*non_abuse_case_num/(sum(trainCategory)*total_num_of_word)
    #Here starts getting p1Vect
    for i in range(numWords):
        total_num_of_word = 0  # total number count of a specific word in trainMatrix
        abuse_case_num = 0 # total number count of a specific word in trainMatrix in abuse conditions
        for j in range(numTrainDocs):
            total_num_of_word += trainMatrix[j][i]
            if trainCategory[j]==1:
                abuse_case_num += trainMatrix[j][i]
        p1Vect[i] = pAbusive*numTrainDocs*abuse_case_num/(sum(trainCategory)*total_num_of_word)
    #print(p0Vect,p1Vect)
    
#
# =================================
    # 返回值:
    # p0Vect长度和词汇集合相等，记录每个词汇出现时该句子属于非侮辱性句子的概率；
    # p1Vect长度和词汇集合相等，记录每个词汇出现时该句子属于侮辱性句子的概率；
    # pAbusive:训练集中侮辱性句子所占比例
    return p0Vect,p1Vect,pAbusive

# 分类方法
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)    
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else: 
        return 0
    
# 测试方法
def testingNB():
    # 获取训练数据
    listOPosts,listClasses = loadDataSet()
    # 创建词汇集合
    myVocabList = createVocabList(listOPosts)
    # 将训练集的句子转换为01数组
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    # 训练分类器
    p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses))

    # 测试1
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print (testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))
    # 测试2
    testEntry = ['stupid', 'garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print (testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))

testingNB()

